In [4]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(영어)', '번역(영-한)', '번역(한-영)', '문서 분류', '용어 식별']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            title_en = line['invention_title_eng']
            title_kr = line['invention_title_kor']
            abstract_en = line['astrt_cont_eng']
            abstract_kr = line['astrt_cont_kor']
            claim_en = line['claim_eng']
            claim_kr = line['claim_kor']
            ipc_number = line['ipc_number']
            keyword_eng = line['tech_word_eng']
            keyword_kor = line['tech_word_kor']
            Indstryclass_name_kor = line['Indstryclass_name_kor']
            ksic_code = line['ksic_code']
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text_kr = {
                '제목' : title_kr,
                '요약' : abstract_kr,
                '청구항' : claim_kr,
                '키워드' : keyword_kor
            }
            text_kr = json.dumps(text_kr, ensure_ascii=False, indent=4)
            data['text'] = text_kr
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
            
            
            #### LM(영어)
            data = {'text': None}
            ## preprocess data from line
            text_en = {
                'Title' : title_en,
                'Abstract' : abstract_en,
                'Claim' : claim_en,
                'Keyword' : keyword_eng
            }
            text_en = json.dumps(text_en, ensure_ascii=False, indent=4)
            data['text'] = text_en
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(영어) end
            
            
            #### 번역(영-한)
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = text_en
            data['output'] = text_kr
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(영-한) end
            
            
            #### 번역(한-영)
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = text_kr
            data['output'] = text_en
            ## preprocess data from line end
            task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역(한-영) end
            
            
            #### 문서 분류
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['title_kr'] = title_kr
            data['input']['abstract_kr'] = abstract_kr
            data['input']['claim_kr'] = claim_kr
            data['input']['title_en'] = title_en
            data['input']['abstract_en'] = abstract_en
            data['input']['claim_en'] = claim_en
            data['output']['ipc_number'] = ipc_number
            data['output']['Indstryclass_name_kor'] = Indstryclass_name_kor
            data['output']['ksic_code'] = ksic_code
            ## preprocess data from line end
            task_files[4].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문서 분류 end
            
            
            #### 용어 식별
            ## preprocess data from line
            data['output']['keyword_kor'] = keyword_kor
            data['output']['keyword_eng'] = keyword_eng
            ## preprocess data from line end
            task_files[5].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 용어 식별 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 5/5 [00:06<00:00,  1.22s/it]
